# Preprocessing

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
image_size = 227
epochs = 50
train_path = os.path.normpath("D:\Projects\DL\BCC\dataset\Train")
test_path = os.path.normpath("D:\Projects\DL\BCC\dataset\Test")
batch_size = 32
seed = 40
color_channels = 3

In [3]:
# Data stored in the form of tensors
train_ds = image_dataset_from_directory(
    train_path,
    labels = "inferred",
    label_mode = "int",
    validation_split = 0.2,
    subset = "training",
    seed = seed,
    image_size = (image_size,image_size),
    batch_size = batch_size,
)

val_ds = image_dataset_from_directory(
    train_path,
    labels = "inferred",
    label_mode = "int",
    validation_split = 0.2,
    subset = "validation",
    seed = seed,
    image_size = (image_size,image_size),
    batch_size = batch_size,
)

Found 2233 files belonging to 2 classes.
Using 1787 files for training.
Found 2233 files belonging to 2 classes.
Using 446 files for validation.


In [4]:
test_ds = image_dataset_from_directory(
    test_path,
    labels = "inferred",
    label_mode = "int",
    seed = seed,
    image_size = (image_size,image_size),
    batch_size = batch_size,
)

Found 638 files belonging to 2 classes.


In [5]:
#kernel crashes when attempting to display some of the images using matplotlib
# Just for displaying some images with their labels
# plt.figure(figsize = (10, 10))
# class_names = train_ds.class_names
# for images, labels in train_ds.take(1):
#     class_labels = [class_names[label] for label in labels.numpy()]
#     for i in range(9):
#         ax = plt.subplot(3, 3, i+1)
#         plt.imshow(images[i].numpy().astype("uint8"))
#         plt.title(class_labels[i])
#         plt.axis("off")

In [6]:
# convert the data to numpy form
train_images, train_labels, val_images, val_labels = [], [], [], []

for images, labels in train_ds:
    train_images.append(images)
    train_labels.append(labels)

train_images = np.vstack(train_images)
train_labels = np.concatenate(train_labels)

for images, labels in val_ds:
    val_images.append(images)
    val_labels.append(labels)

val_images = np.vstack(val_images)
val_labels = np.concatenate(val_labels)


In [7]:
test_images, test_labels = [], []

for images, labels in test_ds:
    test_images.append(images)
    test_labels.append(labels)

test_images = np.vstack(test_images)
test_labels = np.concatenate(test_labels)

In [8]:
print(train_labels.shape)
print(train_images.shape)

(1787,)
(1787, 227, 227, 3)


In [9]:
image_augmentation = ImageDataGenerator(
    rotation_range = 40, # Randomly rotate up to 40 degrees
    width_shift_range = 0.2, # Randomly shift width up to 20%
    height_shift_range = 0.2, # Randomly shift width up to 20%
    shear_range = 0.2, #shear transformations
    zoom_range = 0.2, # zoom in or out up to 20%
    horizontal_flip = True, # Randomly flip images horizontally
    fill_mode = 'nearest', # how to fill in newly created pixels
    featurewise_center = True,
    featurewise_std_normalization = True,
)

In [10]:
image_augmentation.fit(train_images)

In [11]:
train_augmented = image_augmentation.flow(
train_images, train_labels, batch_size = batch_size)
val_augmented = image_augmentation.flow(val_images, val_labels, batch_size = batch_size)

# Model

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, GlobalAveragePooling2D, AveragePooling2D,Flatten

In [13]:
model = Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation="relu", input_shape=(image_size, image_size, 3)))  # Adjust input shape
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))  

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 225, 225, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 223, 223, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [15]:
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer="adam",
              metrics=["accuracy"])

In [16]:
train_steps = len(train_images) // batch_size
valid_steps = len(val_images) // batch_size

In [ ]:
model.fit(train_augmented, steps_per_epoch=train_steps, validation_data=val_augmented, validation_steps=valid_steps, epochs=epochs)

Epoch 1/50
55/55 [==============================] - 156s 3s/step - loss: 0.8096 - accuracy: 0.4957 - val_loss: 0.6933 - val_accuracy: 0.4736
Epoch 2/50
55/55 [==============================] - 151s 3s/step - loss: 0.6931 - accuracy: 0.5060 - val_loss: 0.6936 - val_accuracy: 0.4712
Epoch 3/50
33/55 [=================>............] - ETA: 54s - loss: 0.6928 - accuracy: 0.5157

In [ ]:
results = model.evaluate(test_images, test_labels, batch_size=batch_size)
print( 'Loss = {} and Accuracy = {} %'.format( results[0] , results[1] * 100 ) )

In [ ]:
#End for now